In [81]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [82]:
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [83]:
!pip install pdfplumber

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [84]:
import fasttext
import PyPDF2, pdfplumber
import fastai
import nltk
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import csv
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [85]:
from google.colab import drive
drive.mount('/content/drive')
CV ='/content/drive/My Drive/Colab Notebooks/cc.fr.300.bin'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Chargement du modèle et definition de la fonction de matching soit Similarity

In [86]:
ft_model = fasttext.load_model(CV)

In [87]:
def WordSimilarity(word1, word2): 
   embedding1 = ft_model.get_word_vector(word1)
   embedding2 = ft_model.get_word_vector(word2)
   # Compute the cosine similarity between the embeddings
   similarity_score = np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))
  # print(f"Similarity between {word1} and {word2}: {similarity_score:.2f}")
   return similarity_score 

In [88]:
def Similarity(present_skills, post_skills):
    
  n=0
  
  for x in present_skills:
     List=[]
     for y in post_skills:
        b= WordSimilarity(x,y)
        List.append(b)
        a=max(List)
     if   a>= 0.5:
            n=n+1
     elif a<0.5 and a>= 0.4:
            n=n+0.5
     else:
            n=n+0
  similarity= n*100/len(post_skills)
  return similarity

Traitement de la fiche de description de poste

In [89]:
Req=r"/content/drive/My Drive/Colab Notebooks/OFFRE.pdf"

In [90]:
Req_File=open(Req,'rb')
Script_Req=PyPDF2.PdfReader(Req_File)
pages=len(Script_Req.pages)

In [91]:
Script_Req = []
with pdfplumber.open(Req_File) as pdf:
    for i in range (0,pages):
        page=pdf.pages[i]
        text=page.extract_text()
        print (text)
        Script_Req.append(text)

OFFRE D’EMPLOI : DÉVELOPPEUR(EUSE) WEB
TÂCHES ET RESPONSABILITÉS PRINCIPALES
• Développer le site Web ainsi que des services Web permettant l’affichage optimal et convivial des applications Web pour la diffusion
de données scientifiques ;
• Assurer la documentation et le développement des tests ;
• Participer aux tâches relatives au support technique ;
• Assurer une veille technologique continue.
QUALIFICATIONS ESSENTIELLES
ÉTUDES ET EXPÉRIENCE
• Technique et/ou baccalauréat en informatique ou combinaison acceptable d’études, de formations et/ou d’expériences pertinentes.
• Deux ans d’expérience en développement et intégration Web (comprenant programmation et maintenance).
COMPÉTENCES TECHNIQUES
• Connaissance des langages Web suivants : Javascript, Typescript, HTML5 et CSS ;
• Connaissance de l’architecture MVC ;
• Connaissance d’un langage de programmation orientée objet (ex. JAVA). Le candidat doit maîtriser les concepts et principes
principaux ;
• Connaissance d’un logiciel de gest

In [92]:
Script_Req= ' '.join(Script_Req)# Transform string into list
type(Script_Req)

str

On recupère le contenu de la fiche de description de poste, on effectue la tockenisation de ce contenu et on le met sous forme de liste 

In [93]:
nltk.download('stopwords')
text_tokens = word_tokenize(Script_Req.lower())

tokens_without_sw = [word for word in text_tokens if not word in stopwords.words('french')]

print(tokens_without_sw)
Script_Req = (" ").join(tokens_without_sw)
print(Script_Req)

['offre', '’', 'emploi', ':', 'développeur', '(', 'euse', ')', 'web', 'tâches', 'responsabilités', 'principales', '•', 'développer', 'site', 'web', 'ainsi', 'services', 'web', 'permettant', '’', 'affichage', 'optimal', 'convivial', 'applications', 'web', 'diffusion', 'données', 'scientifiques', ';', '•', 'assurer', 'documentation', 'développement', 'tests', ';', '•', 'participer', 'tâches', 'relatives', 'support', 'technique', ';', '•', 'assurer', 'veille', 'technologique', 'continue', '.', 'qualifications', 'essentielles', 'études', 'expérience', '•', 'technique', 'et/ou', 'baccalauréat', 'informatique', 'combinaison', 'acceptable', '’', 'études', ',', 'formations', 'et/ou', '’', 'expériences', 'pertinentes', '.', '•', 'deux', 'ans', '’', 'expérience', 'développement', 'intégration', 'web', '(', 'comprenant', 'programmation', 'maintenance', ')', '.', 'compétences', 'techniques', '•', 'connaissance', 'langages', 'web', 'suivants', ':', 'javascript', ',', 'typescript', ',', 'html5', 'cs

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [94]:
skills_def = pd.read_csv('/content/drive/My Drive/Colab Notebooks/competences.csv',header=0)

In [95]:
skills_def.head()

,etiquettes
0,3d
1,3d
2,3d-reconstruction
3,importance
4,type de données abstraites


In [96]:
skills_list = list(skills_def['etiquettes'])

In [97]:
#Extraire les compétences présentes dans la fiche de poste
post_skills = []
for skill in skills_list:
    if skill.lower() in Script_Req.lower():
        post_skills.append(skill)
post_skills = list(set(post_skills))

In [98]:
print(post_skills)
Script_Req

['dyn', 'r', 'programmation orientée objet', 'f', 'javascript', 'html', 'communications', 'ava', 'site web', 'avantages', 'fusion', 'wordpress', 'c', 'drupal', 'css', 'postgresql', 'java', 'api', 'diffusion', 'rapide', 'linux', 'sql', 'python', 'git', 'mvc', 'modélisation', 'serveur', 'logiciel']


"offre ’ emploi : développeur ( euse ) web tâches responsabilités principales • développer site web ainsi services web permettant ’ affichage optimal convivial applications web diffusion données scientifiques ; • assurer documentation développement tests ; • participer tâches relatives support technique ; • assurer veille technologique continue . qualifications essentielles études expérience • technique et/ou baccalauréat informatique combinaison acceptable ’ études , formations et/ou ’ expériences pertinentes . • deux ans ’ expérience développement intégration web ( comprenant programmation maintenance ) . compétences techniques • connaissance langages web suivants : javascript , typescript , html5 css ; • connaissance ’ architecture mvc ; • connaissance ’ langage programmation orientée objet ( ex . java ) . candidat doit maîtriser concepts principes principaux ; • connaissance ’ logiciel gestion versions ( git ) ; • connaissance techniques modélisation données systèmes gestion bases 

In [99]:
domaines = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Domaines.csv',header=0)
diplomes = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Diplomes.csv',header=0)

In [100]:
diplomes_list = list(diplomes['Diplome'])
domaines_list = list(domaines['domaine'])

In [101]:
domaines.head()

,domaine
0,design et innovation pour l'environnement
1,scénographe
2,architecte naval
3,architecture et patrimoine contemporain
4,management des projets urbains durables


In [102]:
diplomes.head()

,Diplome
0,Licence
1,Master
2,Doctorat
3,Phd
4,DEA


In [103]:
# Extraire les diplomes présentes dans l'appel d'offre
post_diplomes = []
for diplome in diplomes_list:
    if diplome.lower() in Script_Req.lower():
        post_diplomes.append(diplome)
post_diplomes = list(set(post_diplomes))
print(post_diplomes)

['bac', 'MS', 'BUT', 'CAPA', 'DES', 'CAP', 'baccalauréat']


In [104]:
# Extraire les domaines présentes dans l'appel d'offre
post_domaines = []
for domaine in domaines_list:
    if domaine.lower() in Script_Req:
        post_domaines.append(domaine)
post_domaines = list(set(post_domaines))
print(post_domaines)

['gestion', 'AM', ' programmation', 'communication']


On affiche le contenu de chaque CV

In [105]:
data_dir = '/content/drive/My Drive/Colab Notebooks/CV'

# Vérifier si le dossier existe
if os.path.exists(data_dir):
    # Liste des fichiers dans le répertoire
    files = [f for f in os.listdir(data_dir) if os.path.isfile(os.path.join(data_dir, f))]
    
    # Parcourir les fichiers
    for file in files:
        # Chemin d'accès complet du fichier
        file_path = os.path.join(data_dir, file)
        
        # Vérifier si le fichier est un PDF
        if file.lower().endswith('.pdf'):
            # Ouvrir le fichier PDF
            with open(file_path, 'rb') as CV_File:
                Script = PyPDF2.PdfReader(CV_File)
                pages = len(Script.pages)
                text_list = []
                
                # Extraire le texte de chaque page du PDF
                with pdfplumber.open(CV_File) as pdf:
                    for i in range(pages):
                        page = pdf.pages[i]
                        text = page.extract_text()
                        text_list.append(text)
                
                # Afficher le texte extrait
                print(f"Contenu du fichier {file}:")
                for text in text_list:
                    print(text)
                print('-' * 40)
        else:
            print(f"Le fichier {file} n'est pas un fichier PDF.")
else:
    print("Le dossier spécifié n'existe pas.")


Contenu du fichier cv_sam.pdf:
PROFIL
YAO SAMUEL
Étudiant en 3ème année de licence en
Né le 02/04/2002 systèmes réseaux informatiques et
télécommunications. Créatif, passionné d’
intelligence artificielle et d’électronique, j'ai
toujours eu le goût pour l'apprentissage .
COORDONEES
COORDONEES
FORMATION
01 40957145
Treichville Avenue 21 Rue 15
 Depuis 2020
Samuelyao107@gmail.com
Licence en Systèmes Réseaux
Informatiques et Télécommunications à
ESATIC
COMPETENCE
 2021-2022
S
Certification Développement web
PYTHON  2020-2021
ANGLAIS
Certifié ICDL (International Certification
of Digital Literacy)
Raspberry Pi
 2019-2020
Obtention du BAC C au Lycée Moderne
de Soubré
ARDPYUTINHOO N
EXPERIENCE
MACHINE LEARNING
 Réalisation d’une application
ARDUINO utilisant la vision par ordinateur
ANACONDA
pour la réconnaissance de
monuments ivoiriens
 Réalisation d’une application de
parking intelligent utilisant la
ANGLAIS
ANALYSE DE DONNEES vision par ordinateur pour la
réconnaissance des plaques
d

On effectue le matching de chaque CV avec la fiche de description de poste et on génère un classement décroissant de ces CV en fonction de leur score

In [106]:
data_dir = '/content/drive/My Drive/Colab Notebooks/CV'

# Vérifier si le dossier existe
if os.path.exists(data_dir):
    # Liste des fichiers dans le répertoire
    files = [f for f in os.listdir(data_dir) if os.path.isfile(os.path.join(data_dir, f))]
    
    # Dictionnaire pour stocker les scores
    scores_dict = {}
    
    # Parcourir les fichiers
    for file in files:
        # Chemin d'accès complet du fichier
        file_path = os.path.join(data_dir, file)
        
        # Vérifier si le fichier est un PDF
        if file.lower().endswith('.pdf'):
            # Ouvrir le fichier PDF
            with open(file_path, 'rb') as CV_File:
                Script = PyPDF2.PdfReader(CV_File)
                pages = len(Script.pages)
                text_list = []
                
                # Extraire le texte de chaque page du PDF
                with pdfplumber.open(CV_File) as pdf:
                    for i in range(pages):
                        page = pdf.pages[i]
                        text = page.extract_text()
                        text_list.append(text)
                
                # Convertir la liste de textes en une seule chaîne de caractères
                Script = ' '.join(text_list)
                
                # Appliquer le traitement NLP
                nltk.download('punkt')
                nltk.download('stopwords')
                
                text_tokens = word_tokenize(Script)
                
                tokens_without_sw = [word for word in text_tokens if not word in stopwords.words('french')]
                
                Script = ' '.join(tokens_without_sw)

                
                # Afficher le résultat
                print(f"Contenu du fichier {file}:")
                print(Script)
                
                # Extraire les compétences présentes dans le CV
                present_skills = []
                for skill in skills_list:
                    if skill.lower() in Script.lower():
                        present_skills.append(skill)
                present_skills = list(set(present_skills))
                
                # Afficher les compétences présentes
                print("Compétences présentes :", present_skills)

                # Extraire les diplômes présents dans le CV
                present_diplomes = []
                for diplome in diplomes_list:
                    if diplome.lower() in Script.lower():
                        present_diplomes.append(diplome)
                present_diplomes = list(set(present_diplomes))
                
                # Afficher les diplômes présents
                print("Diplômes présents :", present_diplomes)
                

                # Extraire les domaines présents dans le CV
                present_domaines = []
                for domaine in domaines_list:
                    if domaine.lower() in Script.lower():
                        present_domaines.append(domaine)
                present_domaines = list(set(present_domaines))

                # Afficher les domaines présents
                print("Domaines présents :", present_domaines)
                
                # Calculer les similarités entre les compétences, diplômes et domaines
                skills_similarity = Similarity(present_skills, post_skills)
                diplomes_similarity = Similarity(present_diplomes, post_diplomes)
                domaines_similarity = Similarity(present_domaines, post_domaines)

                 # Afficher les similarités
                print("Similarité des domaines :", domaines_similarity)
                print("Similarité des diplômes :", diplomes_similarity)
                print("Similarité des compétences :", skills_similarity)
                
                
                # Calculer le score final
                score_final = (domaines_similarity + diplomes_similarity + skills_similarity) / 3

                # Afficher le score final
                print("Le score de ce CV est :", score_final)
                
                print('-' * 40)

                # Stocker le score dans le dictionnaire
                scores_dict[file] = score_final
                
    # Trier le dictionnaire des scores par ordre décroissant des valeurs
    sorted_scores = {k: v for k, v in sorted(scores_dict.items(), key=lambda item: item[1], reverse=True)}
    # Afficher le classement
    print("Classement des CV est le suivant:")
    for index, (file, score) in enumerate(sorted_scores.items(), 1):
        print(f"{index}. {file} | Score : {score}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Contenu du fichier cv_sam.pdf:
PROFIL YAO SAMUEL Étudiant 3ème année licence Né 02/04/2002 systèmes réseaux informatiques télécommunications . Créatif , passionné ’ intelligence artificielle ’ électronique , j'ai toujours goût l'apprentissage . COORDONEES COORDONEES FORMATION 01 40957145 Treichville Avenue 21 Rue 15  Depuis 2020 Samuelyao107 @ gmail.com Licence Systèmes Réseaux Informatiques Télécommunications ESATIC COMPETENCE  2021-2022 S Certification Développement web PYTHON  2020-2021 ANGLAIS Certifié ICDL ( International Certification of Digital Literacy ) Raspberry Pi  2019-2020 Obtention BAC C Lycée Moderne Soubré ARDPYUTINHOO N EXPERIENCE MACHINE LEARNING  Réalisation ’ application ARDUINO utilisant vision ordinateur ANACONDA réconnaissance monuments ivoiriens  Réalisation ’ application parking intelligent utilisant ANGLAIS ANALYSE DE DONNEES vision ordinateur réconnaissance plaques ’ immatriculation .
Compétences présentes : ['r', 'développement web', 'arduino', 'électr

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Contenu du fichier cv.pdf:
www.ITjobs.ca Modèle CV Développeur PHP Paul Bertrand 4568 , boulevard Lilas Tél . : 514 000 0000 Courriel : paulbertrand @ webmail.ca Paul Bertrand Développeur PHP OBJECTIF Concevoir développer applications Internet , intranet , gérer besoins réseaux administrer serveurs Linux . COMPÉTENCES Plus trois ans ’ expérience analyse , conception réalisation ’ applications on-line , ainsi ’ gestion réseaux serveurs . Je maîtrise outils développement , design maintenance serveurs Linux Debian . EXPERIENCES PROFESSIONNELLES  2015 – date : XXXXXXXXXXX Poste : Développeur PHP , MySQL , RealBasic , Flash Développement d'applications PHP , MySQL ligne ' e-detailing , e-learning , webinar , web séminaire Administration maintenance serveurs Linux Debian  2015 – date : XXXXXXXXXXX Création magazine XXXXXXX Administration maintenance serveurs Linux Debian . Développements PHP , MySQL , Flash solutions vidéo audio Développement PHP , MySQL site Internet ( basé mambo ) Créati

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
